
#Práctico Métricas RecSys

##Sistemas Recomendadores: IIC3633-1

**Profesor**: Denis Parra

**Ayudantes**:
*   Alejandro Plaza
*   Daniel Sebastian
*   Pablo Messina
*   Carlos Muñoz


---



In [2]:
!pip install pyreclab --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 4.3 MB/s eta 0:00:00


In [3]:
!pip install psutil

In [4]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 37.0 MB/s eta 0:00:00


In [5]:
import pandas as pd
import pyreclab
import numpy as np
import scipy.sparse as sparse
import implicit
import psutil
import time
from sklearn.model_selection import train_test_split

%matplotlib inline

### 2. Calculando métricas básicas en RecSys

###### Carga de datasets

In [7]:
# https://drive.google.com/file/d/1iSmfQILS96JTEgyZn12xlEaay_ybmlLd/view?usp=drive_link
!gdown "1iSmfQILS96JTEgyZn12xlEaay_ybmlLd"

Downloading...
From: https://drive.google.com/uc?id=1iSmfQILS96JTEgyZn12xlEaay_ybmlLd
To: /content/validation_set.csv
100% 134k/134k [00:00<00:00, 63.8MB/s]


In [8]:
# https://drive.google.com/file/d/1lGOYtDD0fXagZ8jaHL6akZMN8caTgzpl/view?usp=drive_link
!gdown "1lGOYtDD0fXagZ8jaHL6akZMN8caTgzpl"

Downloading...
From: https://drive.google.com/uc?id=1lGOYtDD0fXagZ8jaHL6akZMN8caTgzpl
To: /content/training_set.csv
100% 706k/706k [00:00<00:00, 87.2MB/s]


In [9]:
# https://drive.google.com/file/d/1jz8_LLDrtW2Vsn965wMAvhd9owPF2eSu/view?usp=drive_link
!gdown "1jz8_LLDrtW2Vsn965wMAvhd9owPF2eSu"

Downloading...
From: https://drive.google.com/uc?id=1jz8_LLDrtW2Vsn965wMAvhd9owPF2eSu
To: /content/items_styles.csv
100% 19.9k/19.9k [00:00<00:00, 46.5MB/s]


In [10]:
trainfile =  pd.read_csv('training_set.csv')
validfile =  pd.read_csv('validation_set.csv')
items_styles = pd.read_csv('items_styles.csv')

Antes de recomendar siempre hay que realizar un análisis de datos.

In [11]:
trainfile.head()

,userID,itemID,styleID,rating
0,4,54827,1199,4.0
1,1214,10784,1075,4.0
2,5104,48911,1199,3.5
3,1623,16074,1199,4.0
4,1780,19960,1199,5.0


In [12]:
validfile.head()

,userID,itemID,styleID,rating
0,1823,48911,1199,4.0
1,5683,17538,1199,4.0
2,338,11757,1199,4.5
3,6414,1122,395,4.5
4,7115,13350,3268,2.0


###### Matriz de interacciones usuario-ítem

Para este práctico se usará como ejemplo FunkSVD y ALS, sin embargo,pueden realizar el análisis para todos los algoritmos vistos en clases.

In [13]:
user_items = {}
itemset = set()

for row in trainfile.itertuples():
    if row[1] not in user_items:
        user_items[row[1]] = []

    user_items[row[1]].append(row[2])
    itemset.add(row[2])

itemset = np.sort(list(itemset))

sparse_matrix = np.zeros((len(user_items), len(itemset)))

for i, items in enumerate(user_items.values()):
    sparse_matrix[i] = np.isin(itemset, items, assume_unique=True).astype(int)

matrix = sparse.csr_matrix(sparse_matrix.T)

user_item_matrix = matrix.T.tocsr()

original_user_id_to_compact = {key: i for i, key in enumerate(user_items.keys())}
original_item_id_to_compact = {key: i for i, key in enumerate(itemset)}

In [25]:
user_to_min_rating = { u:r for u, r in trainfile.groupby('userID')['rating'].min().items() }
user_to_max_rating = { u:r for u, r in trainfile.groupby('userID')['rating'].max().items() }

###### Entrenamiento modelos RecSys: FunkSVD & ALS

In [14]:
svd = pyreclab.SVD(dataset='training_set.csv', dlmchar=b',', header=False, usercol=0, itemcol=1, ratingcol=3)
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)

In [15]:
als = implicit.als.AlternatingLeastSquares(factors=100, iterations=10)
als.fit(user_item_matrix)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

#### 2.1. RMSE & MAE

Se aprovechará de mostrar un enfoque general, de modo que puedan adaptar fácilmente al proyecto que desarrollan:

In [27]:
import math
from tqdm import tqdm
import random

In [44]:
def sigmoid(x):
    try:
        return 1 / (1 + math.exp(-x))
    except OverflowError:
        if x > 0:
            return 1.0
        else:
            return 0.0

In [45]:
validfile["svd_predict"] = 0.0
validfile["als_predict"] = 0.0

for k in tqdm(range(len(validfile))):
    user_id = validfile["userID"][k]
    item_id = validfile["itemID"][k]
    compact_user_id = original_user_id_to_compact[user_id]
    compact_item_id = original_item_id_to_compact[item_id]

    # Predicción de rating con SVD
    validfile.loc[k, "svd_predict"] = svd.predict(str(user_id), str(item_id))

    # Para estimar rating con ALS, vamos a generar una lista con el item al que queremos predecirle el rating
    # más una muestra aleatoria de otros items, con el fin de tener una noción de la distribución de preferencias del usuario,
    # para luego normalizarla con sigmoid y obtener un rating estimado en el rango de ratings del usuario
    items_to_predict = [compact_item_id]
    items_to_predict.extend(random.sample(range(len(itemset)), 100))
    predictions = als.recommend(compact_user_id, user_item_matrix[compact_user_id], items=items_to_predict, N=101)
    min_score = predictions[1].min()
    max_score = predictions[1].max()
    item_score = predictions[1][predictions[0] == compact_item_id][0]
    min_score = sigmoid(min_score)
    max_score = sigmoid(max_score)
    item_score = sigmoid(item_score)
    min_rating = user_to_min_rating[user_id]
    max_rating = user_to_max_rating[user_id]
    item_rating = (item_score - min_score) / (max_score - min_score) * (max_rating - min_rating) + min_rating
    validfile.loc[k, "als_predict"] = item_rating

y_real     = validfile["rating"].values.tolist()
y_pred_svd = validfile["svd_predict"].values.tolist()
y_pred_als = validfile["als_predict"].values.tolist()

100%|██████████| 7083/7083 [00:18<00:00, 386.99it/s]


In [48]:
# Caso SVD
MSE = np.square(np.subtract(y_real,y_pred_svd)).mean()
RMSE = math.sqrt(MSE)
MAE = np.abs(np.subtract(y_real,y_pred_svd)).mean()

print("Mean squared error    : ", MSE)
print("Root Mean Square Error: ", RMSE)
print("Mean absolute error   : ", MAE)

Mean squared error    :  0.3603823760931761
Root Mean Square Error:  0.6003185621760967
Mean absolute error   :  0.44419790961960187


In [50]:
# Caso ALS
MSE = np.square(np.subtract(y_real,y_pred_als)).mean()
RMSE = math.sqrt(MSE)
MAE = np.abs(np.subtract(y_real,y_pred_als)).mean()

print("Mean squared error    : ", MSE)
print("Root Mean Square Error: ", RMSE)
print("Mean absolute error   : ", MAE)

Mean squared error    :  1.1211383530433703
Root Mean Square Error:  1.0588382090968242
Mean absolute error   :  0.8236424755393589


#### 2.2. MAP, NDCG, Recall

Podemos calcular MAP y NDCG directamente desde su definición oficial, implementadas como funciones.

In [57]:
def precision_at_k(r, k):
    assert 1 <= k <= r.size
    return (np.asarray(r)[:k] != 0).mean()

def average_precision_at_k(r, k):
    r = np.asarray(r)
    score = 0.
    for i in range(min(k, r.size)):
        score += precision_at_k(r, i + 1)
    return score / k

In [58]:
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [65]:
def recall_at_k(relevant_items, recommended_items, k):
    relevant_items = set(relevant_items)
    recommended_items = set(recommended_items[:k])
    intersection = relevant_items.intersection(recommended_items)
    recall = len(intersection) / len(relevant_items) if len(relevant_items) > 0 else 0
    return recall

In [66]:
def evaluate_ranking_SVD_model(model, user_items_test, k):
    mean_ap = 0. # o MAP
    mean_ndcg = 0.
    mean_recall = 0.

    for u in user_items_test.keys():
        rec = model.recommend(str(u), k, includeRated=False) # Pyreclab's API
        rec = [int(x) for x in rec]
        rel_vector = np.isin(rec, user_items_test[u], assume_unique=True).astype(int)
        mean_ap += average_precision_at_k(rel_vector, k)
        mean_ndcg += ndcg_at_k(rel_vector, k, len(user_items_test[u]))
        mean_recall += recall_at_k(user_items_test[u], rec, k)

    mean_ap /= len(user_items_test)
    mean_ndcg /= len(user_items_test)
    mean_recall /= len(user_items_test)

    return mean_ap, mean_ndcg, mean_recall

def evaluate_ranking_ALS_model(model, user_items_test, k):
    mean_ap = 0. # o MAP
    mean_ndcg = 0.
    mean_recall = 0.

    for u in user_items_test.keys():
        rec = model.recommend(u, user_item_matrix[u], k)[0] # implicit's API
        rel_vector = np.isin(rec, user_items_test[u], assume_unique=True).astype(int)
        mean_ap += average_precision_at_k(rel_vector, k)
        mean_ndcg += ndcg_at_k(rel_vector, k, len(user_items_test[u]))
        mean_recall += recall_at_k(user_items_test[u], rec, k)

    mean_ap /= len(user_items_test)
    mean_ndcg /= len(user_items_test)
    mean_recall /= len(user_items_test)

    return mean_ap, mean_ndcg, mean_recall

In [52]:
# This will be used by SVD
user_item_valid = validfile.groupby('userID')['itemID'].apply(list).to_dict()

In [54]:
# This will be used by ALS (remember it uses a compact range)
compact_user_item_valid = {}
for user, items in user_item_valid.items():
    compact_user_item_valid[original_user_id_to_compact[user]] = [original_item_id_to_compact[item] for item in items]

In [55]:
assert len(user_item_valid) == len(compact_user_item_valid)

In [67]:
# SVD
top_k = 10

map, ndcg, recall = evaluate_ranking_SVD_model(svd, user_item_valid, top_k)

print('MAP@{}: {}\nNDCG@{}: {}\nRecall@{}: {}'.format(top_k, map, top_k, ndcg, top_k, recall))

MAP@10: 0.005379334042741862
NDCG@10: 0.007546552937359704
Recall@10: 0.009456050336673679


In [68]:
# ALS
top_k = 10

map, ndcg, recall = evaluate_ranking_ALS_model(als, compact_user_item_valid, top_k)

print('MAP@{}: {}\nNDCG@{}: {}\nRecall@{}: {}'.format(top_k, map, top_k, ndcg, top_k, recall))

MAP@10: 0.054869712689545165
NDCG@10: 0.07826595537013126
Recall@10: 0.08479393375446878


#### 2.4. Diversidad

In [69]:
user_id = 5401 # para SVD
compact_user_id = original_user_id_to_compact[user_id] # para ALS
top_k = 10

svd_ranking = [int(r) for r in svd.recommend(str(user_id), top_k, includeRated=False)]
als_ranking = list(als.recommend(userid=compact_user_id, user_items=user_item_matrix[compact_user_id], N=top_k)[0])
als_ranking = [itemset[r] for r in als_ranking]

print('SVD: Recommendation for user {}: {}'.format(user_id, svd_ranking))
print('ALS: Recommendation for user {}: {}'.format(user_id, als_ranking))

SVD: Recommendation for user 5401: [66849, 47658, 11895, 44081, 72821, 50524, 64487, 15660, 36039, 61745]
ALS: Recommendation for user 5401: [35649, 43091, 961, 37403, 21556, 45260, 1589, 1121, 47659, 45878]


In [80]:
def diversity(recommended_items, item_categories):
    unique_categories = set()
    for item in recommended_items:
        categories = item_categories[item]
        if type(categories) == list:
            unique_categories.update(categories)
        else:
            unique_categories.add(categories)
    return len(unique_categories) / len(recommended_items)

In [81]:
item_categories = items_styles.set_index('itemID')['styleID'].to_dict()
categorias_recomendadas = [item_categories[item] for item in als_ranking]

# Calcular la proporción de categorías únicas
diversidad = diversity(als_ranking, item_categories)
print('Diversidad: {}'.format(diversidad))

Diversidad: 0.7


In [82]:
categorias_recomendadas

[3268, 16386, 365, 1199, 568, 16386, 395, 395, 1199, 14]

#### 2.5. Novedad

Primero se obtiene la popularidad de cada item. Para esto utilizaremos el set de entrenamiento.

In [73]:
item_counts = trainfile['itemID'].value_counts()
total_interacciones = len(trainfile)
items_popularity = item_counts / total_interacciones
items_popularity = items_popularity.to_dict()

In [83]:
def novelty(recommended_items, items_popularity):
    novelty_score = 0
    for item in recommended_items:
        novelty_score += np.log(1 / items_popularity[item])
    return novelty_score

In [84]:
novedad = novelty(als_ranking, items_popularity)
print('Novedad: {}'.format(novedad))

Novedad: 66.55537288016617


### 3. Algunos indicadores de recursos utilizados

#### 3.1. Tiempo de Ejecución

In [85]:
# Iniciar el temporizador
start_time = time.time()

# Entrenamiento del modelo
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)

# Detener el temporizador
end_time = time.time()

print(f"Tiempo de entrenamiento: {end_time - start_time} segundos")

Tiempo de entrenamiento: 3.3651838302612305 segundos


#### 3.2. Memoria utilizada

In [86]:
# Obtener información de memoria antes del entrenamiento
mem_info_before = psutil.virtual_memory().used

# Entrenamiento del modelo
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)

# Obtener información de memoria después del entrenamiento
mem_info_after = psutil.virtual_memory().used

# Calcular el uso de memoria durante el entrenamiento
memory_used = (mem_info_after - mem_info_before) / (1024 ** 2)  # Convertir a MB

print(f"Memoria utilizada: {memory_used} MB")


Memoria utilizada: 16.984375 MB


#### 3.3. Uso de CPU

In [87]:
# Obtener el uso de CPU antes del entrenamiento
cpu_usage_before = psutil.cpu_percent(interval=None)

# Entrenamiento del modelo
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)

# Obtener el uso de CPU después del entrenamiento
cpu_usage_after = psutil.cpu_percent(interval=None)

print(f"Uso de CPU: {cpu_usage_after - cpu_usage_before}%")

Uso de CPU: 28.299999999999997%
